Author: Shailendra Chalasani
Intention: Supervised Learning Likes to dislikes ratio - KNN

In [23]:
import gzip
import json
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import pandas as pd
from collections import Counter, defaultdict
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
df_video_dislikes = pd.read_csv(f"../../data/cleaned_datasets/df_video_dislikes.csv")
print(len(df_video_dislikes))
df_video_dislikes.dropna(inplace=True)
print(len(df_video_dislikes))

37422
37264


In [25]:
df_video_dislikes.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'video_id', 'title', 'published_at',
       'view_count', 'likes_count', 'dislikes_count', 'comment_count', 'tags',
       'description', 'comments', 'topic1', 'topic2', 'topic3'],
      dtype='object')

In [26]:
# df_video_dislikes = df_video_dislikes[['title','view_count','likes_count','dislikes_count','comment_count','tags','description','comments']]
df_video_dislikes = df_video_dislikes[['view_count','likes_count','dislikes_count','comment_count','topic1','topic2','topic3']]
# df_video_dislikes['likes_dislikes_ratio'] = df_video_dislikes['likes_count']/df_video_dislikes['dislikes_count']
df_video_dislikes

,view_count,likes_count,dislikes_count,comment_count,topic1,topic2,topic3
0,1048888,19515,226,1319,2,9,3
1,15352638,359277,7479,18729,5,7,1
2,925281,11212,401,831,1,2,9
3,2641597,39131,441,3745,5,3,7
4,715724,32887,367,1067,5,0,1
...,...,...,...,...,...,...,...
37417,120408275,2180780,35871,81360,5,0,8
37418,2841917,20759,1049,2624,6,9,7
37419,13346678,720854,4426,90616,1,7,8
37420,252057,19198,1234,1471,9,8,0


In [27]:
from sklearn.preprocessing import StandardScaler

X = df_video_dislikes[['view_count','likes_count','comment_count','topic1','topic2','topic3']]
y =df_video_dislikes['dislikes_count']

train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=0)

scaler = StandardScaler().fit(train_X)
X_train = scaler.transform(train_X)
X_test = scaler.transform(test_X)

In [28]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
import numpy as np

# Create the KNeighborsRegressor object
knn = KNeighborsRegressor()

# Define the parameter grid for grid search
param_grid = {
    'n_neighbors': [3, 5, 7,9,11,13,15,17,19,21,23,25],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, scoring='r2', cv=5)

# Fit the GridSearchCV object to the data
grid_search.fit(train_X, train_y)

# Get the mean and standard deviation of the R2 scores
r2_scores = grid_search.cv_results_['mean_test_score']
r2_mean = np.mean(r2_scores)
r2_std = np.std(r2_scores)

# Get the best parameters
best_params = grid_search.best_params_

print("Mean R2 Score:", r2_mean)
print("Standard Deviation of R2 Score:", r2_std)
print("Best Parameters:", best_params)


Mean R2 Score: 0.43595942103077184
Standard Deviation of R2 Score: 0.038349126747694015
Best Parameters: {'metric': 'manhattan', 'n_neighbors': 25, 'weights': 'distance'}


In [29]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.inspection import permutation_importance


# Train the KNeighborsRegressor model
knn = KNeighborsRegressor(n_neighbors=25, metric='manhattan', weights='distance')
knn.fit(train_X, train_y)

# Calculate permutation importances
result = permutation_importance(knn, test_X, test_y, n_repeats=10)

# Get the feature importances and their indices
importances = result.importances_mean
indices = np.argsort(importances)

# Print the feature importances
for f in range(train_X.shape[1]):
    print(f"{train_X.columns[indices[f]]}: {importances[indices[f]]}")



topic3: -7.44894580906319e-08
topic2: 3.7521667384243784e-08
topic1: 4.0703306791733953e-08
comment_count: 0.0006088099176504325
likes_count: 0.03365021487976334
view_count: 0.4624124622734917


In [30]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

# Define the range of values for the number of neighbors (hyperparameter)
neighbors = [3, 5, 7,9,11,13,15,17,19,21,23,25]

# Perform sensitivity analysis by iterating over different values of neighbors
for k in neighbors:
    # Instantiate the KNN regressor with the current number of neighbors
    model = KNeighborsRegressor(n_neighbors=k)

    # Evaluate the model using cross-validation and the desired metric (e.g., mean squared error)
    scores = cross_val_score(model, train_X, train_y, cv=5, scoring='neg_mean_squared_error')

    # Calculate the mean and standard deviation of the scores
    print(scores)
    mean_score = -scores.mean()  # Since scoring='neg_mean_squared_error', we take the negative of the mean score
    std_score = scores.std()

    # Print the results
    print(f"Number of neighbors: {k}, Mean MSE: {mean_score:.5f}, Standard Deviation: {std_score:.5f}")


[-7.23591148e+08 -2.67652550e+08 -4.90521414e+08 -3.69771281e+08
 -6.05461653e+08]
Number of neighbors: 3, Mean MSE: 491399609.18012, Standard Deviation: 162345328.28145
[-7.20529883e+08 -2.58108833e+08 -5.62927748e+08 -3.20880058e+08
 -5.97993445e+08]
Number of neighbors: 5, Mean MSE: 492087993.39273, Standard Deviation: 174630087.09031
[-7.20967384e+08 -2.67681840e+08 -5.83329225e+08 -3.00150512e+08
 -4.83733979e+08]
Number of neighbors: 7, Mean MSE: 471172588.07516, Standard Deviation: 170757942.51390
[-7.56380205e+08 -2.39081656e+08 -6.07273435e+08 -2.82303472e+08
 -4.68714038e+08]
Number of neighbors: 9, Mean MSE: 470750561.41666, Standard Deviation: 194632845.34154
[-7.89498753e+08 -2.14860830e+08 -5.90607392e+08 -2.87146984e+08
 -4.50154835e+08]
Number of neighbors: 11, Mean MSE: 466453758.71384, Standard Deviation: 207599503.49428
[-8.15546059e+08 -2.20666266e+08 -5.66863621e+08 -2.65321012e+08
 -4.58434465e+08]
Number of neighbors: 13, Mean MSE: 465366284.29301, Standard Devia